In [40]:
import pickle
import numpy as np
from itertools import product

# Dados com as probabilidades do stacking para o caso 1 (interseção) fold 1.

### Utiliza-se o caso 1 fold 1 como exemplo, pois ele tem instâncias onde os modelos erram e o stacking acerta. Alguns folds não existem estas instâncias.

In [34]:
data = np.load("logs/stacking_data/1/1/data.npz", allow_pickle=True)

In [23]:
list(data.keys()), data["X_train"].shape, data["X_test"].shape, data["y_train"].shape, data["y_test"].shape

(['X_train', 'X_test', 'y_train', 'y_test', 'probas'],
 (6783, 96),
 (695, 96),
 (6783,),
 (695,))

# Todas as classes da base.

### Apesar de que para alguns dias algumas classes estão ausentes, as probabilidades geradas pelo Luca possuem todas as classes, sendo zero para as classes ausentes em todas as instâncias.

In [41]:
probs_cols = [
    "censys",
    "driftnet",
    "internetcensus",
    "intrinsec",
    "ipip",
    "mirai",
    "onyphe",
    "rapid7",
    "securitytrails",
    "shadowserver",
    "shodan",
    "u_mich",
    "unk_bruteforcer",
    "unk_exploiter",
    "unk_spammer",
    "unknown",
]
probs_cols.sort()

# Predições onde o stacking acertou e os modelos erraram.

In [24]:
with open("logs/filtered_preds.pkl", "rb") as fd:
    filtered = pickle.load(fd)

In [25]:
filtered.keys(), filtered[1].keys(), filtered[1][1].keys()

(dict_keys([1, 2, 3, 4, 5]),
 dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 dict_keys(['X', 'y', 'idxs', 'stacking_probas', 'columns_labels', 'model']))

## Acessando os dados do fold 0 do caso 1 (interseção)

IDs onde o stacking acertou e os modelos não.

In [26]:
filtered[1][1]["idxs"]

array([504, 505])

Probabilidades do teste do stacking. Igual ao que está no X_test da variável data no início do notebook.

In [21]:
filtered[1][1]["stacking_probas"].shape

(695, 16)

Probabilidades dos modelos (Equivalente ao X_test ou stacking_probas indexados por filtered[1][1]["idxs"])

In [28]:
filtered[1][1]["X"].shape

(2, 96)

Label das instâncias (Equivalente ao y_test ou stacking_probas indexados por filtered[1][1]["idxs"])

In [30]:
filtered[1][1]["y"].shape

(2,)

Meta-modelo do stacking (para esse dia todas as classes estavam presentes. Pode ser que para alguns dias não estejam)

In [33]:
filtered[1][1]["model"], filtered[1][1]["model"].classes_, filtered[1][1]["model"].classes_.shape

(LogisticRegression(max_iter=500, n_jobs=-1),
 array(['censys', 'driftnet', 'internetcensus', 'intrinsec', 'ipip',
        'mirai', 'onyphe', 'rapid7', 'securitytrails', 'shadowserver',
        'shodan', 'u_mich', 'unk_bruteforcer', 'unk_exploiter',
        'unk_spammer', 'unknown'], dtype=object),
 (16,))

Labels de cada dimensão do vetor de probabilidades (probabilidades dos modelos base concatnadas) utilizados para inserir no stacking.

Os labels são organizados da seguinte forma: fonte-modelo-classe.

In [35]:
filtered[1][1]["columns_labels"]

['darknet-features-censys',
 'darknet-features-driftnet',
 'darknet-features-internetcensus',
 'darknet-features-intrinsec',
 'darknet-features-ipip',
 'darknet-features-mirai',
 'darknet-features-onyphe',
 'darknet-features-rapid7',
 'darknet-features-securitytrails',
 'darknet-features-shadowserver',
 'darknet-features-shodan',
 'darknet-features-u_mich',
 'darknet-features-unk_bruteforcer',
 'darknet-features-unk_exploiter',
 'darknet-features-unk_spammer',
 'darknet-features-unknown',
 'darknet-idarkvec-censys',
 'darknet-idarkvec-driftnet',
 'darknet-idarkvec-internetcensus',
 'darknet-idarkvec-intrinsec',
 'darknet-idarkvec-ipip',
 'darknet-idarkvec-mirai',
 'darknet-idarkvec-onyphe',
 'darknet-idarkvec-rapid7',
 'darknet-idarkvec-securitytrails',
 'darknet-idarkvec-shadowserver',
 'darknet-idarkvec-shodan',
 'darknet-idarkvec-u_mich',
 'darknet-idarkvec-unk_bruteforcer',
 'darknet-idarkvec-unk_exploiter',
 'darknet-idarkvec-unk_spammer',
 'darknet-idarkvec-unknown',
 'darknet-ig

No caso 1 (interseção) são duas fontes e três modelos. Total de dimensões do vetor de probabilidade: 2 * 3 * 16 = 96. Quando for uma única fonte o número de dimensões será a metada.

As probabilidades foram carregadas em ordem alfabética fonte - modelo.

In [39]:
list(product(["darknet", "honeypot"], ["features", "idarkvec", "igcngru_features"]))

[('darknet', 'features'),
 ('darknet', 'idarkvec'),
 ('darknet', 'igcngru_features'),
 ('honeypot', 'features'),
 ('honeypot', 'idarkvec'),
 ('honeypot', 'igcngru_features')]

Extraindo a probababilidade de cada modelo para a primeira instância onde o stacking acertou e os modelos erraram do caso 1 fold 1.

### IDX 504

In [44]:
n_classes = len(probs_cols)

In [50]:
models_probs = {}
for count, (source, model_name) in enumerate(product(["darknet", "honeypot"], ["features", "idarkvec", "igcngru_features"])):
    models_probs[f"{source}-{model_name}"] = data["X_test"][504][count * n_classes : (count + 1) * n_classes]


### Rápido teste de sanidade.

In [71]:
TEST_ID = 504
for key in models_probs:
    print(f"Modelo: {key}",
        "Predição: ", probs_cols[models_probs[key].argmax()])

filtered[1][1]["idxs"][0], "Predição stacking: ", filtered[1][1]["model"].classes_[
    filtered[1][1]["stacking_probas"][TEST_ID].argmax()
], "Label: ", data["y_test"][TEST_ID]

Modelo: darknet-features Predição:  unknown
Modelo: darknet-idarkvec Predição:  unknown
Modelo: darknet-igcngru_features Predição:  unknown
Modelo: honeypot-features Predição:  unknown
Modelo: honeypot-idarkvec Predição:  unknown
Modelo: honeypot-igcngru_features Predição:  unknown


(504, 'Predição stacking: ', 'unk_spammer', 'Label: ', 'unk_spammer')